# Chapter 2: Static Analysis with mypy

Professional Python development uses static type checking with mypy to catch errors before runtime. This notebook covers mypy configuration, common patterns, and advanced type checking strategies.

## Section 1: mypy Fundamentals

In [ ]:
# mypy catches type errors that would only appear at runtime

def greet(name: str) -> str:
    """Greet someone."""
    return f"Hello, {name}!"

# Static type check (mypy would catch this):
# greet(123)  # Error: Argument 1 to "greet" has incompatible type "int"; expected "str"

# But Python allows it at runtime
result = greet("Alice")
print(result)

In [ ]:
# mypy requires type hints to perform checking
from typing import List, Dict, Optional

def process_items(items: List[int]) -> int:
    """Process a list of integers."""
    return sum(items)

# mypy checks this usage
numbers = [1, 2, 3, 4, 5]
total = process_items(numbers)
print(f"Total: {total}")

# Without type hints, mypy can't check
def process_anything(items):  # No type hint
    return sum(items if isinstance(items, list) else [])

result = process_anything([1, 2, 3])
print(f"Result: {result}")

## Section 2: Common Type Checking Patterns

In [ ]:
from typing import Optional, Union

# Pattern 1: Optional values
def get_user(user_id: int) -> Optional[str]:
    """Get a user or None if not found."""
    users = {1: "Alice", 2: "Bob"}
    return users.get(user_id)

# mypy requires checking for None
user = get_user(1)
if user is not None:
    greeting = f"Hello, {user}!"
    print(greeting)
else:
    print("User not found")

In [ ]:
# Pattern 2: Union types
def process(value: Union[int, str]) -> str:
    """Process int or str differently."""
    if isinstance(value, int):
        return str(value * 2)
    else:
        return value.upper()

# mypy knows the type after isinstance check
print(process(42))
print(process("hello"))

In [ ]:
# Pattern 3: Generics with constraints
from typing import TypeVar, Generic

T = TypeVar('T')
Comparable = TypeVar('Comparable', int, float, str)

def first(items: list[T]) -> T:
    """Get first item, preserving type."""
    return items[0]

def maximum(a: Comparable, b: Comparable) -> Comparable:
    """Return the maximum of two comparable values."""
    return a if a > b else b  # type: ignore

print(f"first([1,2,3]) = {first([1,2,3])}")
print(f"maximum(5, 3) = {maximum(5, 3)}")
print(f"maximum('apple', 'zebra') = {maximum('apple', 'zebra')}")

## Section 3: Type Comments and Annotations

In [ ]:
# Type: ignore suppresses mypy errors for specific lines
def unsafe_operation(x: int) -> str:
    # mypy would complain about mixed types, but we know what we're doing
    return x + "hello"  # type: ignore

# In a real project, this would fail mypy check without the comment
# But at runtime it still fails:
try:
    result = unsafe_operation(5)
except TypeError as e:
    print(f"Runtime error (as expected): {e}")

In [ ]:
# Cast: explicitly tell mypy what type something is
from typing import cast, Any

def process_json(data: Any) -> str:
    """Process JSON data."""
    # We know this is a string, but mypy sees Any
    text = cast(str, data)
    return text.upper()

# Works at runtime
result = process_json("hello")
print(f"Result: {result}")

# But if actual type is wrong, cast doesn't help
try:
    bad_result = process_json(123)
except AttributeError as e:
    print(f"Runtime error: {e}")
    print("Cast doesn't protect at runtime!")

## Section 4: isinstance() for Type Narrowing

In [ ]:
# Use isinstance to narrow types safely
from typing import Union

def handle_value(value: Union[int, str, list]) -> str:
    """Handle different types."""
    
    # Type narrowing with isinstance
    if isinstance(value, int):
        # mypy knows value is int here
        return f"Integer: {value * 2}"
    
    elif isinstance(value, str):
        # mypy knows value is str here
        return f"String: {value.upper()}"
    
    else:
        # mypy knows value is list here
        return f"List with {len(value)} items"

print(handle_value(42))
print(handle_value("hello"))
print(handle_value([1, 2, 3]))

In [ ]:
# Type narrowing with hasattr (structural typing)
from typing import Protocol

class Closeable(Protocol):
    def close(self) -> None:
        ...

def maybe_close(obj: object) -> None:
    """Close an object if it has close() method."""
    if hasattr(obj, 'close') and callable(getattr(obj, 'close')):
        # Type narrowing doesn't work perfectly here
        # But we can use cast for clarity
        closeable = cast(Closeable, obj)
        closeable.close()

class FileWrapper:
    def close(self) -> None:
        print("Closing file")

wrapper = FileWrapper()
maybe_close(wrapper)
maybe_close("string")  # Has no close(), does nothing

## Section 5: Assertions and Type Checking

In [ ]:
# Assertions document assumptions
from typing import Optional

def process_user(user: Optional[str]) -> str:
    """Process user (must exist)."""
    # Assert that user is not None
    assert user is not None, "User must not be None"
    
    # After assertion, mypy knows user is str, not Optional[str]
    return user.upper()

# Valid usage
result = process_user("alice")
print(f"Result: {result}")

# What if we pass None?
try:
    bad_result = process_user(None)
except AssertionError as e:
    print(f"Assertion failed: {e}")

In [ ]:
# Use assertions to narrow types within function
def safe_divide(x: int, y: int) -> float:
    """Divide x by y safely."""
    assert y != 0, "Cannot divide by zero"
    return x / y

print(f"10 / 2 = {safe_divide(10, 2)}")

try:
    result = safe_divide(10, 0)
except AssertionError as e:
    print(f"Assertion failed: {e}")

## Section 6: Any Type - Use Sparingly

In [ ]:
from typing import Any

# Any disables type checking
def process_any(value: Any) -> Any:
    """Process any value (no type safety)."""
    if isinstance(value, int):
        return value * 2
    return str(value)

# mypy can't check these
print(process_any(5))
print(process_any("hello"))
print(process_any([1, 2, 3]))

print("\nAny allows anything - avoid when you can be specific!")

In [ ]:
# Good practice: be specific,avoid Any
from typing import Union, TypeVar, Sequence

# Bad: uses Any
def bad_process(items: list[Any]) -> Any:
    return len(items)

# Good: specific types
def good_process(items: Sequence[int]) -> int:
    return len(items)

# Better: generic
T = TypeVar('T')
def better_process(items: Sequence[T]) -> int:
    return len(items)

print(f"bad_process([1,2,3]) = {bad_process([1, 2, 3])}")
print(f"good_process([1,2,3]) = {good_process([1, 2, 3])}")
print(f"better_process([1,2,3]) = {better_process([1, 2, 3])}")

## Summary

### mypy Configuration
In `pyproject.toml`:
```toml
[tool.mypy]
python_version = "3.12"
strict = true
warn_unused_ignores = true
ignore_missing_imports = true
```

### Type Checking Strategies
1. **Type hints on all functions**: `def func(x: int) -> str:`
2. **isinstance() checks**: Narrow Union types safely
3. **Optional handling**: Check for None before use
4. **Type narrowing**: mypy understands control flow
5. **Generics**: Use TypeVar for flexible types
6. **Protocol**: Define structural interfaces

### Advanced Tools
- `cast()`: Explicitly tell mypy the type
- `type: ignore`: Suppress specific errors
- `assert`: Runtime + static type narrowing
- `TypeGuard`: Custom type narrowing functions

### Best Practices
1. Never use `Any` when you can be specific
2. Always check `Optional` before use
3. Use `isinstance()` to narrow union types
4. Enable `strict` mode in mypy configuration
5. Run mypy in CI/CD pipeline before testing